<a href="https://colab.research.google.com/github/Smart-Adaptive-Intelligent-Systems-Lab/mouse_trajectory/blob/main/LSTM_Mouse_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Prediction for the mouse movement 
The goal of the project is to predict whoever is using the computer by predicting his movement and try to fit a user, but right here I'm just trying to predict the mouse movement of a the cursor

### Import


In [ ]:
import cv2
import numpy as np
import pickle
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import ModelCheckpoint
import csv
import pandas as pd
import time

### Algorithm parameters

In [ ]:
dataset_filename = 'mouse_data.txt'
weights_save_name = "weights_lstm_mouse.hdf5"
CONTINUE_TRAINING_WHERE_YOU_LEFT_OFF=True
timesteps = 16
n_predictions = 128
MAX_X,MAX_Y = 1024,1024

### Global Variables
Initialize the global variables that we will be using after

In [ ]:
img = []
dataset = []
testing_dataset = []
model = None
model_generated = False
state = 'idle'
time_data = []

### Saving and Loading the dataset from a file
Just a function to save and load the dataset that we use

In [ ]:
# Save and Load dataset from file
def save_dataset():
    with open(dataset_filename, "wb") as fp:
        pickle.dump(dataset, fp)

def load_dataset():
    global dataset
    with open(dataset_filename, "rb") as fp:
        dataset = pickle.load(fp)

### Normalize and de-normalize

Normalize data_points in our dataset

In [ ]:
def normalize(data_points,scale_down=True):
    if(scale_down==True):
        data_points[:] = [[float(data[0])/MAX_X, float(data[1])/MAX_Y] for data in data_points]
    else:
        data_points[:] = [[int(data[0]*MAX_X), int(data[1]*MAX_Y)] for data in data_points]

### Mouse callback function

This is the predictive model

In [ ]:
def trace_mouse_movements(event,x,y,flags,param):
    if event == cv2.EVENT_MOUSEMOVE:
        if(state == 'record_data'):
            cv2.circle(img,(x,y),2,(255,0,0),-1)
            end = time.time()
            dataset.append([x,y])
            time_data.append(end-start)
        elif(state == 'predict'):
            img.fill(0)
            testing_dataset.append([x,y])
            for [_x,_y] in testing_dataset:
                cv2.circle(img,(_x,_y),2,(0,127,255),-1)
            if(len(testing_dataset)>timesteps):
                testing_dataset.pop(0)
                pred_dataset = [i for i in testing_dataset]
                normalize(pred_dataset,scale_down=True)
                for i in range(n_predictions):
                    input_sequence = np.array([j for j in pred_dataset[i:i+timesteps]])
                    input_sequence = np.reshape(input_sequence,(-1,2,input_sequence.shape[0]))
                    _temp = model.predict(input_sequence)
                    pred_dataset.append([_temp[0][0],_temp[0][1]])
                normalize(pred_dataset,scale_down=False)
                for [_x,_y] in pred_dataset[timesteps:]:
                    cv2.circle(img,(_x,_y),2,(255,0,0),-1)
        elif(state == 'idle'):
            cv2.circle(img,(x,y),2,(int(np.random.randint(0,255)),int(np.random.randint(0,255)),int(np.random.randint(0,255))),-1)


### Loading weight

Loading the weight from file if the file already exist

In [ ]:
def load_pretrained_weights():
    model
    try:
        model.load_weights(weights_save_name)
    except:
        print('Pre-trained weights do not exist. Please train model to obtain weights')


### Graph the model
Here we are just using simple layers such as LSTM and hidden layers aswell

In [ ]:
def generate_model(load_weights=False):
    model = Sequential()
    model.add(LSTM(64, return_sequences=False, input_shape=(2,timesteps)))
    #model.add(LSTM(32, return_sequences=True))
    #model.add(LSTM(32))
    model.add(Dense(2, activation='relu'))
    model.compile(loss='mse', optimizer='adam')
    if(load_weights==True): load_pretrained_weights()
    model_generated = True

### Generator for data

In [ ]:
def initialize_data_xy():
    load_dataset()
    normalize(dataset,scale_down=True)
    xy_coord = [i for i in dataset]
    n = len(xy_coord)-1
    while(True):
        i = np.random.randint(0,n)
        try:
            x_data = xy_coord[i:i+timesteps]
            y_data = xy_coord[i+timesteps]
            yield x_data, y_data
        except:
            pass

### Fetch a dataset

In [ ]:
def get_data_to_train(how_many):
    data = initialize_data_xy()
    input_data = []
    output_data = []
    for i in range(how_many):
        _temp = data.next()
        input_data.append(_temp[0])
        output_data.append(_temp[1])
    input_data = np.array(input_data)
    output_data = np.array(output_data)
    input_data = np.reshape(input_data,(input_data.shape[0],2,input_data.shape[1]))
    return input_data,output_data


### Training

In [ ]:
# Training...
def train(load_weights=False):
    global model
    x_train, y_train = get_data_to_train(1000)
    if(model_generated==False):
        generate_model(load_weights=load_weights)
    elif(load_weights==True):
        global model
        model.load_weights(weights_save_name)
    callbacks_list = [ModelCheckpoint(weights_save_name, monitor='loss', verbose=1, save_best_only=True, mode='auto', save_weights_only='True')]
    model.fit(x_train, y_train, batch_size=1000, epochs=5000, verbose=2, callbacks=callbacks_list)


### Helper function
Some help to know which input you can use

In [ ]:
# Helper function
def print_help():
    print('''Predict mouse movements using Long Short Term Memory(LSTM) Network
    Usage:
    Press \'i\' for idle mode (mouse points are plotted in different colours)
    Press \'r\' to start recording data, press \'r\' again to save dataset (mouse points are plotted red)
    Press \'t\' to start training using the dataset (mouse are not plotted)
    Press \'p\' to predict mouse movements (actual movements are plotted orange, predictions are plotted blue)
    Press \'c\' to clear screen
    Press \'h\' to show this help script
    Press \'ESC\' to quit
    Notes:''')


### Main function

In [ ]:
img = np.zeros((MAX_X,MAX_Y,3), np.uint8)
cv2.namedWindow('image')
cv2.setMouseCallback('image',trace_mouse_movements)
print_help()
start = time.time()
while(1):
    cv2.imshow('image',img)
    ip = cv2.waitKey(20) & 0xFF
    if ip == 27:
        break
    elif(ip==ord('r')):
        if(state != 'record_data'):
            state = 'record_data'
            print('Started recording data')
        else:
            save_dataset()
            print('Saved recorded data')
    elif(ip==ord('p')):
        if(model_generated==True):
            load_pretrained_weights()
        else:
            generate_model(load_weights=True)
        testing_dataset = []
        state = 'predict'
    elif(ip==ord('t')):
        state = 'train'
        print('Training started')
        train(load_weights=CONTINUE_TRAINING_WHERE_YOU_LEFT_OFF)
        print('Training done')
        state = 'idle'
    elif(ip==ord('i')):
        state = 'idle'
    elif(ip==ord('c')):
        img.fill(0)
    elif(ip==ord('h')):
        print_help()
cv2.destroyAllWindows()

Predict mouse movements using Long Short Term Memory(LSTM) Network
    Usage:
    Press 'i' for idle mode (mouse points are plotted in different colours)
    Press 'r' to start recording data, press 'r' again to save dataset (mouse points are plotted red)
    Press 't' to start training using the dataset (mouse are not plotted)
    Press 'p' to predict mouse movements (actual movements are plotted orange, predictions are plotted blue)
    Press 'c' to clear screen
    Press 'h' to show this help script
    Press 'ESC' to quit
    Notes:
Started recording data
Saved recorded data


In [ ]:
load_dataset()
xy_coord = [i for i in dataset]
header = ['time_stamp','x_coord','y_coord']
with open('mouse_data.csv','w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)

    for i in range(len(xy_coord)-1):
        writer.writerow([time_data[i],xy_coord[i][0],xy_coord[i][1]])


In [ ]:
df = pd.read_csv("mouse_data.csv")
df.head()

,time_stamp,x_coord,y_coord
0,0.0,443,446
1,0.0,444,446
2,0.0,447,446
3,0.0,456,443
4,0.0,458,441
